# Simple RAG Chatbot using Langchain

### Table of Contents

1. **Load LLM**
2. **Create Prompt Teamplate**
3. **Merge Prompt Template & LLM to create Chain**
4. **Load External Data**
5. **Generate Embeddings**
6. **Build Index in Vector Store (using Embeddings)**
7. **Create Retrieval Chain**

### Installation

* * **pip install langchain**

## 1 Load LLM

In [1]:
from langchain_community.llms import Ollama

llm = Ollama(model="llama2") # base_url = 'http://localhost:11434'

llm

Ollama()

In [2]:
response = llm.invoke("Do you know about Claude 3?")

print(response)

I'm not familiar with a "Claude 3." Could you please provide more context or information about who or what Claude 3 is? That will help me better understand your question and provide a more accurate response.


## 2 Create Prompt Template

In [3]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an Artificial Intelligence News Reporter."),
    ("user", "{query}")
])

## 3 Merge Prompt Template & LLM to Create LangChain Chain

In [4]:
llm_app = prompt | llm

llm_app

ChatPromptTemplate(input_variables=['query'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are an Artificial Intelligence News Reporter.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], template='{query}'))])
| Ollama()

In [5]:
response = llm_app.invoke({"query": "Do you know about Claude 3?"})

print(response)


Ah, a fellow tech enthusiast! Yes, I'm afraid I don't have any information on a mysterious figure known as "Claude 3." As an AI news reporter, my sources are constantly updating and refreshing, but I haven't come across any credible leads or intel regarding this individual. Can you tell me more about who Claude 3 is and what they're up to? Perhaps a whistleblower or insider might have some juicy details for me! 😏


## 4 Load External Data

* **pip install beautifulsoup4**

In [6]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.anthropic.com/news/claude-3-family")

docs = loader.load()

In [7]:
urls = [
        "https://www.anthropic.com/news/releasing-claude-instant-1-2",
        "https://www.anthropic.com/news/claude-pro",
        "https://www.anthropic.com/news/claude-2",
        "https://www.anthropic.com/news/claude-2-1",
        "https://www.anthropic.com/news/claude-2-1-prompting",
        "https://www.anthropic.com/news/claude-3-family",
        "https://www.anthropic.com/claude"
       ] 

docs = []
for url in urls:
    loader = WebBaseLoader(url)
    docs.extend(loader.load())
    
len(docs)

7

In [8]:
docs

[Document(page_content='Releasing Claude Instant 1.2 \\ AnthropicClaudeAPIResearchCompanyNewsCareersProductReleasing Claude Instant 1.2Aug 9, 2023●1 min readBusinesses working with Claude can now access our latest version of Claude Instant, version 1.2, available through our API.\xa0Claude Instant is our faster, lower-priced yet still very capable model, which can handle a range of tasks including casual dialogue, text analysis, summarization, and document comprehension.Claude Instant 1.2 incorporates the strengths of our latest\xa0model Claude 2\xa0in real-world use cases and shows significant gains in key areas like math, coding, reasoning, and safety. It generates longer, more structured responses and follows formatting instructions better. Instant 1.2 also shows improvements in quote extraction, multilingual capabilities, and question answering.Claude Instant 1.2 outperforms Claude Instant 1.1 on math and coding, achieving 58.7% on the Codex evaluation compared to 52.8% in our prev

## 5 Generate Embeddings

In [9]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings_llm = OllamaEmbeddings(model="llama2") # base_url = 'http://localhost:11434'

In [10]:
embeddings = embeddings_llm.embed_query("How are you?")

embeddings[:5]

[-0.5058066844940186,
 -0.31813421845436096,
 2.4193434715270996,
 0.2437400221824646,
 -0.11038058251142502]

In [11]:
type(embeddings), len(embeddings)

(list, 4096)

In [12]:
embeddings = embeddings_llm.embed_documents([
                                "Claude 3 is latest Conversational AI Model from Anthropic.",
                                "Gemini is latest Conversational AI Model from Google.",
                                "Llama-2 is latest Conversational AI Model from Meta.",
                                "Mixtral is latest Conversational AI Model from Mistral AI.",
                                "GPT-4 is latest Conversational AI Model from OpenAI."
                               ])

len(embeddings), type(embeddings), len(embeddings[0])

(5, list, 4096)

## 6 Build Index in Vector Store

* **pip install faiss-cpu**

In [13]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()

documents = text_splitter.split_documents(docs)

len(documents)

12

In [14]:
from langchain_community.vectorstores import FAISS

vector_index = FAISS.from_documents(documents, embeddings_llm)

vector_index

There are 3 functions to create index.

* **from_documents()**
* **from_embeddings()**
* **from_texts()**

In [15]:
retriever = vector_index.as_retriever()

relevant_docs = retriever.invoke({"input": "Do you know about Claude 3?"})

len(relevant_docs)

4

In [16]:
relevant_docs

[Document(page_content='Releasing Claude Instant 1.2 \\ AnthropicClaudeAPIResearchCompanyNewsCareersProductReleasing Claude Instant 1.2Aug 9, 2023●1 min readBusinesses working with Claude can now access our latest version of Claude Instant, version 1.2, available through our API.\xa0Claude Instant is our faster, lower-priced yet still very capable model, which can handle a range of tasks including casual dialogue, text analysis, summarization, and document comprehension.Claude Instant 1.2 incorporates the strengths of our latest\xa0model Claude 2\xa0in real-world use cases and shows significant gains in key areas like math, coding, reasoning, and safety. It generates longer, more structured responses and follows formatting instructions better. Instant 1.2 also shows improvements in quote extraction, multilingual capabilities, and question answering.Claude Instant 1.2 outperforms Claude Instant 1.1 on math and coding, achieving 58.7% on the Codex evaluation compared to 52.8% in our prev

In [17]:
for doc in relevant_docs:
    print(f"Title : {doc.metadata['title']}, Source: {doc.metadata['source']}")

Title : Releasing Claude Instant 1.2 \ Anthropic, Source: https://www.anthropic.com/news/releasing-claude-instant-1-2
Title : Long context prompting for Claude 2.1 \ Anthropic, Source: https://www.anthropic.com/news/claude-2-1-prompting
Title : Introducing the next generation of Claude \ Anthropic, Source: https://www.anthropic.com/news/claude-3-family
Title : Introducing Claude Pro \ Anthropic, Source: https://www.anthropic.com/news/claude-pro


In [18]:
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f500b86f850>)

## 7 Create Retrieval Chain

### 7.1 Create History Aware Retreiver

In [19]:
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from langchain.chains import create_history_aware_retriever

chat_history = [HumanMessage(content="Do you know about Claude 3?"),
                AIMessage(content="Yes, I am well aware of Claude 3 AI conversational bot from Anthropic which has 3 models (Opus, Haiku & Sonnet). Please provide more context info on how can I help you.")]

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])

retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

relevant_docs = retriever_chain.invoke({
    "input": "Tell me about different models in detail.",
    "chat_history": chat_history
})

In [20]:
for doc in relevant_docs:
    print(f"Title : {doc.metadata['title']}, Source: {doc.metadata['source']}")

Title : Introducing the next generation of Claude \ Anthropic, Source: https://www.anthropic.com/news/claude-3-family
Title : Long context prompting for Claude 2.1 \ Anthropic, Source: https://www.anthropic.com/news/claude-2-1-prompting
Title : Introducing the next generation of Claude \ Anthropic, Source: https://www.anthropic.com/news/claude-3-family
Title : Claude \ Anthropic, Source: https://www.anthropic.com/claude


### 7.2 Create Retrieval Chain

In [21]:
from langchain_core.prompts import MessagesPlaceholder
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

system_prompt = """
Answer the user's questions based on the below context and your internal knowledge.
Give priority to context and if you are not sure then say you are not aware of topic:

<context>
{context}
</context>
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])

#document_chain  = prompt | llm
document_chain = create_stuff_documents_chain(llm, prompt)


retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f500b86f850>))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[MessagesPlaceholder(variable_name='chat_history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Given the abo

In [22]:
response = retrieval_chain.invoke({"input": "Tell me about different models in detail.", 
                                   "chat_history": chat_history})

type(response)

dict

In [23]:
response.keys()

dict_keys(['input', 'chat_history', 'context', 'answer'])

In [24]:
print(response["answer"])

AI: Of course! The Claude 3 model family is a group of three state-of-the-art language models developed by Anthropic. These models are designed to excel across various cognitive tasks and have demonstrated significant advancements over previous models in terms of intelligence, capability, and performance. Here's a detailed overview of each model in the Claude 3 family:

1. Claude 3 Haiku: This is the most capable but less intelligent model in the family, with a context window of 200K tokens. It can perform tasks such as task automation, plan and execute complex actions, and interactive coding. While it's not as intelligent or capable as the other two models, Haiku is still an excellent choice for a wide range of applications.
2. Claude 3 Sonnet: The second model in the family has a context window of 500K tokens. Sonnet demonstrates higher intelligence and capability than Haiku, with the ability to process longer inputs and provide more accurate responses. It excels in R&D tasks such as

In [25]:
for doc in response["context"]:
    print(f"Title : {doc.metadata['title']}, Source: {doc.metadata['source']}")

Title : Releasing Claude Instant 1.2 \ Anthropic, Source: https://www.anthropic.com/news/releasing-claude-instant-1-2
Title : Claude \ Anthropic, Source: https://www.anthropic.com/claude
Title : Claude 2 \ Anthropic, Source: https://www.anthropic.com/news/claude-2
Title : Introducing the next generation of Claude \ Anthropic, Source: https://www.anthropic.com/news/claude-3-family


In [26]:
chat_history.append(HumanMessage(content="Tell me about different models in detail."))
chat_history.append(AIMessage(content=response["answer"]))

chat_history

[HumanMessage(content='Do you know about Claude 3?'),
 AIMessage(content='Yes, I am well aware of Claude 3 AI chatbot from Anthropic. Please provide more context info on how can I help you.'),
 HumanMessage(content='Tell me about different models in detail.'),
 AIMessage(content="AI: Of course! The Claude 3 model family is a group of three state-of-the-art language models developed by Anthropic. These models are designed to excel across various cognitive tasks and have demonstrated significant advancements over previous models in terms of intelligence, capability, and performance. Here's a detailed overview of each model in the Claude 3 family:\n\n1. Claude 3 Haiku: This is the most capable but less intelligent model in the family, with a context window of 200K tokens. It can perform tasks such as task automation, plan and execute complex actions, and interactive coding. While it's not as intelligent or capable as the other two models, Haiku is still an excellent choice for a wide rang

In [27]:
response = retrieval_chain.invoke({"input": "Tell me more about Claude 3 Opus.", 
                                   "chat_history": chat_history})

print(response["answer"])

'AI: Of course! Claude 3 Opus is the most intelligent model in the Claude 3 family, designed to excel in advanced cognitive tasks that require a deep understanding of human language and reasoning. With a context window of 1 million tokens, Opus can navigate open-ended prompts and unseen scenarios with remarkable fluency and human-like comprehension.\n\nHere are some key features and capabilities of Claude 3 Opus:\n\n1. Natural Language Classification and Sentiment Analysis: Opus can accurately classify text into various categories, such as positive/negative sentiment, topic classification, and entity recognition. It can also perform multi-step reasoning to understand the nuances of language and context.\n2. Drug Discovery and Development: Opus can assist in the discovery of new drugs by analyzing large datasets of chemical compounds, predicting potential side effects, and identifying optimal drug combinations.\n3. Strategy Analysis and Decision-Making: Opus can analyze complex strategi

## Summary

In this video, I explained how to create simple **RAG** chatbot application using **langchain**. Feel free to let me know your views and doubts in comments section.